# Imports


In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np

# Allows for further data manipulation and analysis
import pandas as pd
import matplotlib.pyplot as plt  # Plotting
import matplotlib.dates as mdates  # Styling dates

import datetime as dt

import time

import yfinance as yf

# Used for file handling like deleting files
# and get data from a directory
import os
from os import listdir
from os.path import isfile, join

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
# Use plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

import warnings

warnings.simplefilter("ignore")


# Constants


In [3]:
PATH = "C:\\Users\\Eduardo Nuñez\\OneDrive\\Python\\Derek Banas\\Investing\\Whilshire_Stocks\\"

# Start end date defaults
S_DATE = "2017-02-01"
E_DATE = "2022-12-06"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)


# Get Dataframe from CSV


In [4]:
# A dataframe is a pandas type of object used like a spreadsheet
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + ".csv", index_col=0)
    except FileNotFoundError:
        print("File doesn't exist")
    else:
        return df


# Plot Bollinger Bands


In [5]:
def plot_with_boll_bands(df: pd.DataFrame, ticker):
    fig = go.Figure()

    candle = go.Candlestick(
        x=df.index, open=df["Open"], high=df["High"], low=df["Low"], close=df["Close"], name="Candlestick"
    )

    upper_line = go.Scatter(
        x=df.index, y=df["upper_band"], line=dict(color="rgba(250,0,0,0.75)", width=1), name="Upper Band"
    )

    mid_line = go.Scatter(
        x=df.index, y=df["middle_band"], line=dict(color="rgba(0,0,250,0.75)", width=0.7), name="Middle Band"
    )

    lower_line = go.Scatter(
        x=df.index, y=df["lower_band"], line=dict(color="rgba(0,250,0,0.75)", width=1), name="Lower Band"
    )

    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(mid_line)
    fig.add_trace(lower_line)

    fig.update_xaxes(title="Date", rangeslider_visible=True)
    fig.update_yaxes(title="Price")

    fig.update_layout(title=ticker + " Bollinger Bands", height=800, width=1200, showlegend=True)
    fig.show()


# Plot Ichimoku


In [6]:
def get_fill_color(label):
    if label >= 1:
        return "rgba(0,250,0,0.4)"  # Returns green
    else:
        return "rgba(250,0,0,0.4)"  # Returns red


In [7]:
def get_ichimoku(df: pd.DataFrame, ticker):
    # fig = go.Figure()

    candle = go.Candlestick(
        x=df.index, open=df["Open"], high=df["High"], low=df["Low"], close=df["Close"], name="Candlestick"
    )

    df1 = df.copy()
    fig = go.Figure()
    # in this case the fill should be green (value = 1 in get_fill_color)
    df["label"] = np.where(df["SpanA"] > df["SpanB"], 1, 0)
    df["group"] = df["label"].ne(df["label"].shift()).cumsum()
    df = df.groupby("group")

    dfs = []
    for name, data in df:
        dfs.append(data)

    for df in dfs:
        fig.add_traces(go.Scatter(x=df.index, y=df.SpanA, line=dict(color="rgba(0,0,0,0)")))
        fig.add_traces(
            go.Scatter(
                x=df.index,
                y=df.SpanB,
                line=dict(color="rgba(0,0,0,0)"),
                fill="tonexty",
                fillcolor=get_fill_color(df["label"].iloc[0]),
            )
        )

    baseline = go.Scatter(x=df1.index, y=df1["Baseline"], line=dict(color="pink", width=2), name="Baseline")
    conversion = go.Scatter(x=df1.index, y=df1["Conversion"], line=dict(color="black", width=1), name="Conversion")
    lagging = go.Scatter(x=df1.index, y=df1["Lagging"], line=dict(color="purple", width=2), name="Lagging")
    span_a = go.Scatter(x=df1.index, y=df1["SpanA"], line=dict(color="green", width=2, dash="dot"), name="Span A")
    span_b = go.Scatter(x=df1.index, y=df1["SpanB"], line=dict(color="red", width=1, dash="dot"), name="Span B")

    fig.add_trace(candle)
    fig.add_trace(baseline)
    fig.add_trace(conversion)
    fig.add_trace(lagging)
    fig.add_trace(span_a)
    fig.add_trace(span_b)

    fig.update_layout(title=ticker + " Ichimoku",height=800, width=1200, showlegend=True)

    fig.show()


# Plots


In [8]:
ticker = "AAPL"
test_df = get_stock_df_from_csv(ticker)
# print(test_df["Open"].rolling(window=5).mean())
# print(test_df["Open"])  # prints the column index (Date) and column "Open"
# test_df["label"] = np.where(test_df["SpanA"] > test_df["SpanB"], 1, 0)
# print(test_df[["SpanA", "SpanB", "label"]])
# print(test_df["label"].ne(test_df["label"].shift()))
# test_df["group"] = test_df["label"].ne(test_df["label"].shift()).cumsum()
# print(test_df[["SpanA", "SpanB", "label", "group"]])

# print(test_df[["Open", "Close", "label"]])
# print(test_df[["Open", "Close", "label"]].shift())
# print(test_df["label"])
plot_with_boll_bands(test_df, ticker)
get_ichimoku(test_df, ticker)
